In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install tsplib95

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.21.0 requires tabulate>=0.9, but you have tabulate 0.8.10 which is incompatible.


In [ ]:
import tsplib95  # Importer la bibliothèque tsplib95
import random  # Importer la bibliothèque random
import numpy as np  # Importer la bibliothèque numpy sous le nom np
import time  # Importer la bibliothèque time

# Charger l'instance TSP à partir d'un fichier
problem = tsplib95.load(r'/content/drive/MyDrive/Colab Notebooks/tsplib/eil101.tsp')

# Obtenir le nombre de nœuds dans l'instance TSP
n = len(problem.node_coords)

# Définir une fonction pour calculer la distance totale d'un parcours
def calculate_tour_distance(tour):
    total_distance = 0
    for i in range(len(tour)):
        total_distance += problem.get_weight(tour[i - 1], tour[i])
    return total_distance

# Définir une fonction pour générer un parcours initial aléatoire
def generate_random_tour():
    tour = list(range(1, n+1))
    random.shuffle(tour)
    return tour

# Définir une fonction pour effectuer une recherche locale 2-opt
def two_opt_local_search(initial_tour):
    current_tour = initial_tour[:]
    best_tour = current_tour[:]
    best_distance = calculate_tour_distance(best_tour)

    improved = True
    while improved:
        improved = False
        for i in range(1, n - 1):
            for j in range(i + 1, n):
                new_tour = current_tour[:]
                new_tour[i:j+1] = reversed(new_tour[i:j+1])  # Effectuer l'échange 2-opt
                new_distance = calculate_tour_distance(new_tour)
                if new_distance < best_distance:
                    best_tour = new_tour[:]
                    best_distance = new_distance
                    improved = True
        current_tour = best_tour[:]

    return best_tour, best_distance

# Définir une fonction pour calculer les statistiques
def calculate_statistics(results):
    optimum = min(results)
    best_solution = min(results)
    worst_solution = max(results)
    moyenne = sum(results) / len(results)
    ecart_type = np.std(results)
    c1 = results.count(optimum)
    c2 = len(results) - c1
    return optimum, best_solution, worst_solution, moyenne, ecart_type, c1, c2

# Définir le nombre d'itérations
num_iterations = 10

# Exécuter l'algorithme plusieurs fois
execution_times = []
solutions = []

for _ in range(num_iterations):
    start_time = time.time()
    initial_tour = generate_random_tour()
    best_tour, best_distance = two_opt_local_search(initial_tour)
    end_time = time.time()
    execution_time = end_time - start_time
    execution_times.append(execution_time)
    solutions.append(best_distance)

# Calculer les statistiques
optimum, best_solution, worst_solution, moyenne, ecart_type, c1, c2 = calculate_statistics(solutions)

# Afficher les informations
print("Optimum:", optimum)
print("Meilleure solution:", best_solution)
print("Pire solution:", worst_solution)
print("Moyenne:", moyenne)
print("Écart type:", ecart_type)
print("c1 (nombre de fois où l'optimum est atteint):", c1)
print("c2 (nombre de fois où l'optimum n'est pas atteint):", c2)
print("Temps d'exécution (s):", sum(execution_times))


Optimum: 672
Best solution: 672
Worst solution: 702
Moyenne: 685.5
Ecart type: 8.309632964216892
c1 (nbr de fois optimum atteint): 1
c2 (nbr de fois optimum n’est pas atteint): 9
Temps d'exécution (s): 1843.0573263168335


In [ ]:
import numpy as np
import time

def qap_obj(a, b, p):
    """
    Calcule la valeur objective d'une solution pour le QAP.

    Args:
    - a (numpy.ndarray): Matrice des flux.
    - b (numpy.ndarray): Matrice des distances.
    - p (numpy.ndarray): Permutation.

    Returns:
    - float: Valeur objective.
    """
    return np.sum(a * b[p[:, None], p])

def qap_local_search(a, b, max_iter=1000):
    """
    Effectue une recherche locale pour le QAP en utilisant 2-opt.

    Args:
    - a (numpy.ndarray): Matrice des flux.
    - b (numpy.ndarray): Matrice des distances.
    - max_iter (int): Nombre maximum d'itérations.

    Returns:
    - numpy.ndarray: Meilleure permutation trouvée.
    - float: Valeur objective de la meilleure permutation.
    - list: Liste des coûts rencontrés pendant la recherche.
    """
    n = len(a)
    if n == 0:
        return np.array([]), 0, []

    best_p = np.random.permutation(n)
    best_cost = qap_obj(a, b, best_p)

    costs = [best_cost]
    for _ in range(max_iter):
        i, j = np.random.choice(n, 2, replace=False)
        new_p = best_p.copy()
        new_p[i], new_p[j] = new_p[j], new_p[i]
        new_cost = qap_obj(a, b, new_p)
        if new_cost < best_cost:
            best_p = new_p
            best_cost = new_cost
        costs.append(new_cost)

    return best_p, best_cost, costs

# Charger les données QAP depuis bur26a.dat
with open(r'/content/drive/MyDrive/Colab Notebooks/qaplib/tai50a.dat', 'r') as f:
    lines = f.readlines()

data = [list(map(int, line.split())) for line in lines if line.strip()]
n = data[0][0]
a = np.array(data[1:n+1])
b = np.array(data[n+1:])

start_time = time.time()

# Effectuer une recherche locale pour le QAP
meilleure_permutation, meilleur_cout, couts = qap_local_search(a, b)

end_time = time.time()
execution_time = end_time - start_time

cout_moyen = np.mean(couts)
ecart_type = np.std(couts)

print("Meilleure permutation :", meilleure_permutation)
print("Meilleur coût :", meilleur_cout)
print("Coût moyen :", cout_moyen)
print("Écart type :", ecart_type)
print("Temps d'exécution :", execution_time, "secondes")


Meilleure permutation : [18 30  1 28 48 26 44  7 31  2 45  3 32 15 35 36 34 25 43 13 17  4 41 37
 42 46  6  0 19  9 40 20 23 24 38 21 12 14 10  8 16 47 27 39 33 11 49 22
 29  5]
Meilleur coût : 5283370
Coût moyen : 5458184.367632368
Écart type : 111208.64162186916
Temps d'exécution : 0.09538006782531738 secondes
